In [3]:
import sys
import os.path
import numpy as np
from landlab import imshow_grid
from landlab import RasterModelGrid
from matplotlib import pyplot as plt

#use the utility functions
utilspath=os.path.join("../","util")
sys.path = [utilspath]+sys.path
import myLandlabTools as tools

topopath=os.path.join("../","topographies")

%matplotlib inline

In [4]:
demDict = {
    'valley2' : ['valley1.98_[100,100,10][2.83,0.00,-2.83,270.00].asc'],
    'valley5' : ['valley4.96_[100,100,10][7.07,0.00,-7.07,270.00].asc'],
    'valley10' : ['valley10.00_[100,100,10][14.14,0.00,-14.14,270.00].asc'],
    'valley15' : ['valley15.20_[100,100,10][21.21,0.00,-21.21,270.00].asc'],
    'valley20' : ['valley20.64_[100,100,10][28.28,0.00,-28.28,270.00].asc'],
    'valley25' : ['valley26.41_[100,100,10][35.36,0.00,-35.36,270.00].asc'],
 }

# soil infiltration  
# from Gowdish and Munoz & Carsel and Parrish
hydCondDict = {
            'cond1' :[1.7e-7, 'clay'],
            'cond2' :[2.7e-7, 'silty clay'],
            'cond3' :[3.3e-7, 'sandy clay'],
            'cond4' :[5.5e-7, 'silty clay loam'],
            'cond5' :[7.2e-7, 'clay loam'],           
            'cond6' :[8.3e-7, 'sandy clay loam'],
            'cond7' :[1.8e-6, 'silt loam' ],          
            'cond8' :[3.6e-6, 'loam'],               
            'cond9' :[6.1e-6, 'sandy loam' ],         
            'cond10':[1.6e-5, 'loamy sand'],      
}


roughnessDict = {
    'clean': [0.03],
    'mediummix': [0.05],
    'overgrown': [0.07],
}

# Storm [starting_precip_mmhr, storm duration s]
#
# A constant precipitation rate can be passed to the OverlandFlow class, where precipitation persists for the 
# entire model run. Alternatively, a single event can be set within the time loop, and then water can drain 
# from the system when the precipitation event is over.
stormDict = {
    'high':[30., 17100.],
    'low':[15., 17100.],
    'intense': [50., 900.],
    'intense1': [50., 3600.],
    'intense2': [100., 900.],
    '1995-11-18':[15.6, 17100.],
    '1997-03-11':[34.8, 4500.],
    '2009-02-10':[18.0, 7200.],
    '2010-12-08':[16.8, 5400.],
    '2012-11-24':[16.8, 3900],
}


In [5]:
# set simulation parameters - use the dictionaries in the previous block to set up the scenario
     
# storm
storm_flag = 'intense1'             

# Water column height above the surface previously absorbed into the soil.
# Do NOT set to zero
soilWaterInfiltrationDepth = 0.1

# Depth of water above the surface - a value close to 0 is recommended here
surfaceWaterDepth = 1e-8

# simulation run time [hours]
runHours = 3.      
# interval to report progress [hours]
reportInterval = 1.  

# Storm  
starting_precip_mmhr = stormDict[storm_flag][0]
storm_duration = stormDict[storm_flag][1] 

# Manning n
deltaM = 0.01
manningRange = np.arange(roughnessDict['clean'][0],roughnessDict['overgrown'][0]+deltaM, deltaM)

# Hydraulic conductivity
hydcondRange = []
soilType = []
for key in hydCondDict:
    hydcondRange.append(hydCondDict[key][0])
    soilType.append(hydCondDict[key][1])
    
soilInf = True

In [6]:
# Loop over all combinations of input parameters

# For each dem in the list
for key in demDict:
    dem = demDict[key][0]
    print(dem)
    
    # set boundary and monitor link
    
    if 'gully' in key:
        boundary = 'noData'               
        outletNode = 296
        monitorLink = 590 
    elif 'valley' in key:
        boundary = 'setSelf'  
        outletNode = 9898
        monitorLink = 19500      # links for node 9898 : 19600 19699 19599 19500    
    else:
        if 'flat' in key:
            boundary = 'allOpen'   
            outletNode = 465
            monitorLink = 870      # links for node 465 : 900 929 899 870 
        else:
            boundary = 'rightOpen'  
            outletNode = 449
            monitorLink = 854      
            
    # for each hydraulic conductivity
    for min_h, soil_type in zip(hydcondRange,soilType):

        # for each roughness value 
        for mannings_n in manningRange:

            rmg, hydraulic_conductivity, discharge_at_outlet, hydrograph_time, h, d = tools.runModel(
                                    topopath, dem, 
                                    soil_type, min_h, min_h, 
                                    starting_precip_mmhr, storm_duration, 
                                    soilWaterInfiltrationDepth, surfaceWaterDepth,
                                    mannings_n,
                                    boundary, monitorLink,  
                                    runHours, reportInterval,
                                    soilInf = soilInf, sinkFiller = False, 
                                    showProgress = False, showPlots = False,
                                    outlet_node = outletNode)

            fileName = tools.getLLFilename(key,mannings_n,min_h,soilWaterInfiltrationDepth)
            tools.writeLLData(fileName,hydrograph_time,discharge_at_outlet,h,d)


valley1.98_[100,100,10][2.83,0.00,-2.83,270.00].asc
valley4.96_[100,100,10][7.07,0.00,-7.07,270.00].asc
valley10.00_[100,100,10][14.14,0.00,-14.14,270.00].asc
valley15.20_[100,100,10][21.21,0.00,-21.21,270.00].asc
valley20.64_[100,100,10][28.28,0.00,-28.28,270.00].asc
valley26.41_[100,100,10][35.36,0.00,-35.36,270.00].asc
